# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

# Initial Setup

To begin I've chosen to focus on getting all of the stats from one game.

Due to personal preference I've chosen to look at the preseason game between the Dallas Fuel and the Houston Outlaws. The page on winstons lab for the game can be found here:

[Preseason: Fuel vs. Outlaws](https://www.winstonslab.com/matches/match.php?id=2367)

The individual match id for the site is 2367. We'll try to keep everything as modular as possible during prototyping.

In [2]:
BASE_MATCH_URL = "https://www.winstonslab.com/matches/match.php"

In [3]:
match_id = "2367"

In [4]:
full_url = BASE_MATCH_URL + "?id=" + match_id

## Get the base html using requests

In [8]:
response = requests.get(full_url)
html = BeautifulSoup(response.text, "lxml")

# Overview stats

In [38]:
div = html.findAll("div", {"class": "side-by-side-stats"})[0] # table containing detailed stats
detail = div.findAll("div", recursive=False)[2]

In [39]:
detail.text

''

Ok so this is empty because It loads the data after clicked detailed stats. I guess we'll have to use Selenium and PhantomJS

In [5]:
from selenium import webdriver
import time

In [6]:
driver = webdriver.Chrome("/Applications/chromedriver")

JK using Chrome. I need to find a chrome headless driver.

In [7]:
driver.get(full_url)
time.sleep(5)

In [8]:
detail_button = driver.find_element_by_class_name("showDetailedStatsBut")

In [9]:
detail_button.click()

In [55]:
detail_stats = driver.find_element_by_id("detailedStatsDiv_0")

In [10]:
page_source = driver.page_source

In [11]:
new_html = BeautifulSoup(page_source, "lxml")

In [12]:
div = new_html.findAll("div", {"class": "side-by-side-stats"})[0] # table containing detailed stats
detail = div.findAll("div", recursive=False)[2]

In [13]:
stat_table = detail.table

In [14]:
table_headers = stat_table.thead
table_data = stat_table.tbody

In [15]:
header_elems = table_headers.findAll("th")

In [16]:
header_list = []
for header in header_elems:
    header_list.append(header.text)
pprint(header_list)

['Player',
 'Hero',
 'Time',
 'Rating',
 'FWin%',
 'PTK',
 'K/10',
 'D/10',
 'U/10',
 'TTCU',
 'KPU',
 'UE',
 'UOOF',
 'FK',
 'FD',
 'Impact',
 'Elim/10',
 'dmg/10',
 'heal/10',
 'ObjTime/10',
 'ObjKills/10']


In [17]:
row_list = table_data.findAll("tr")

In [19]:
for row in row_list[0:3]:
    elems = row.findAll("td")
    print("-----------------------")
    for i, item in enumerate(elems[0:]):
        print("{}: {}".format(header_list[i], item.text.strip()))

-----------------------
Player: Bani
Hero: Zenyatta
Time: 00:08:14
Rating: 1451
FWin%: 55.56%
PTK: 19.15%
K/10: 10.93
D/10: 6.07
U/10: 4.86
TTCU: 91
KPU: 0
UE: 44.44
UOOF: 25%
FK: 22%
FD: 11%
Impact: 1355
Elim/10: N/A
dmg/10: N/A
heal/10: N/A
ObjTime/10: N/A
ObjKills/10: N/A
-----------------------
Player: Boink
Hero: Mercy
Time: 00:44:32
Rating: 1206
FWin%: 55.32%
PTK: 0.56%
K/10: 0.22
D/10: 3.82
U/10: 3.59
TTCU: 87
KPU: 0
UE: 21.6
UOOF: 12.5%
FK: 0%
FD: 9%
Impact: 1116
Elim/10: N/A
dmg/10: N/A
heal/10: N/A
ObjTime/10: N/A
ObjKills/10: N/A
-----------------------
Player: Taimou
Hero: Roadhog
Time: 00:30:25
Rating: 1193
FWin%: 57.14%
PTK: 25.77%
K/10: 8.22
D/10: 4.6
U/10: 4.27
TTCU: 74
KPU: 0
UE: 5.36
UOOF: 8.33%
FK: 21%
FD: 11%
Impact: 1098
Elim/10: N/A
dmg/10: N/A
heal/10: N/A
ObjTime/10: N/A
ObjKills/10: N/A


### Function
We need to put this part into a function.

In [ ]:
# function for grabbing detailed match data from a particular match
def detailed_match_data(game_id):
    
    